In [1]:
%config IPCompleter.greedy=True

In [2]:
import sys
import os
import pandas as pd
import cv2
import csv
sys.path.append("../..") # relative path to module toolkit

In [3]:
from keras.preprocessing.image import img_to_array
from keras.models import load_model
from keras.backend import clear_session
import numpy as np
import matplotlib.pyplot as plt
from toolkit import getLabelsFromDir
from sklearn.metrics import classification_report
%matplotlib inline

Using TensorFlow backend.


tf.estimator package not installed.
tf.estimator package not installed.


In [4]:
def classification_report_csv(report, label):
    report_data = []
    lines = report.split('\n')
    for line in lines[2:-4]:
        row = {}
        row_data = line.split(' ')
        row_data = [data for data in row_data if data]
        if len(row_data) > 0:
            row['class'] = row_data[0]
            row['precision'] = float(row_data[1])
            row['recall'] = float(row_data[2])
            row['f1_score'] = float(row_data[3])
            row['support'] = float(row_data[4])
            report_data.append(row)
    dataframe = pd.DataFrame.from_dict(report_data)
    dataframe.to_csv('./classification_report/classification_report_'+ label +'.csv', index = False)

In [5]:
target_size = (224, 224)
genus_model = load_model("./resNet50pretrained.h5")

In [8]:
import os

cut_out_path = "./cutout"
if not os.path.exists(cut_out_path):
    os.makedirs(cut_out_path)

cr_out_path = "./classification_report"
if not os.path.exists(cr_out_path):
    os.makedirs(cr_out_path)
    
genus_val_dir = "../../../images/images_genus_shuffled/val/"

genus_labels = getLabelsFromDir(genus_val_dir)

all_genus_predict = []
all_y_true = []
all_y_pred = []

for cutout_species in [
    "Bisnius nigriventris",
    "Gabrius piliger",
    "Lathrobium castaneipenne",
    "Lathrobium longulum",
    "Lathrobium pallidipenne",
    "Neobisnius procerulus",
    "Philonthus pseudovarians",
    "Philonthus spinipes",
    "Philonthus umbratilis",
    "Platydracus chalcocephalus",
    "Quedius invreae",
    "Quedius nemoralis",
    "Rugilus geniculatus",
    "Rugilus similis",
    "Scopaeus minimus",
    "Tasgius pedator",
    "Tasgius winkleri",
    "Tetartopeus rufonitidus"
    ]:
    
    cutout_species_dir_path = "../../../images/cutout_species/" + cutout_species
    genus_predicts = []
    y_true = []
    y_pred = []

    for file in os.listdir(cutout_species_dir_path):
        if file.endswith(".jpg"):
            file_path = os.path.join(cutout_species_dir_path, file)

            real_genus = file_path.split(" ")[0].split("/")[-1]
            real_species = file_path.split("/")[-1].split("_")[0]

            img = cv2.imread(file_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            resized_img = cv2.resize(img, target_size) 

            img_array = img_to_array(resized_img) / 255.0
            img_array = img_array.reshape((1,) + img_array.shape)
            genus_Y_pred = genus_model.predict(img_array, batch_size=None, verbose=0, steps=None)
            genus_y_pred = np.argmax(genus_Y_pred, axis=1)

            predicted_genus = genus_labels[genus_y_pred[0]]
            
            obj = {"correct_prediction": predicted_genus == real_genus, "filepath": file_path, "real_genus": real_genus, "real_species": real_species, "predicted_genus": predicted_genus}
            
            y_true.append(real_genus)
            all_y_true.append(real_genus)
            y_pred.append(predicted_genus)
            all_y_pred.append(predicted_genus)
            
            genus_predicts.append(obj)
            all_genus_predict.append(obj)
    
    report = classification_report(y_true, y_pred)
    classification_report_csv(report, cutout_species)
    
    keys = genus_predicts[0].keys()
    with open('./cutout/' + cutout_species + '_results.csv', 'w') as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(genus_predicts)

   
report = classification_report(all_y_true, all_y_pred)
classification_report_csv(report, 'overall')

keys = all_genus_predict[0].keys()
with open('./cutout/overall_results.csv', 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(all_genus_predict)

        
        
        